In [1]:
import pandas as pd
import spacy
from spacy.tokens import Doc, DocBin
from pathlib import Path
import json


2023-12-16 13:20:47.891487: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-12-16 13:20:47.899452: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-12-16 13:20:47.948204: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-12-16 13:20:47.948961: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-12-16 13:20:48.567770: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Co

In [2]:
DATA_PATH = "/home/user/MPIT_HOMEWORK/data/data.json"
SPACY_DATA_OUT_PATH = "/home/user/MPIT_HOMEWORK"

In [3]:
df = pd.read_json(DATA_PATH)

In [4]:
df

,текстовое_сообщение,тип_события,дополнительные_параметры
0,Создай задачу: разработать дизайн логотипа,создание_задачи,{'текст_задачи': 'разработать дизайн логотипа'}
1,Создай заметку: идеи для нового проекта,создание_заметки,{'текст_заметки': 'идеи для нового проекта'}
2,Создай задачу: подготовить отчет к конференции,создание_задачи,{'текст_задачи': 'подготовить отчет к конферен...
3,Создай заметку: список дел на неделю,создание_заметки,{'текст_заметки': 'список дел на неделю'}
4,Создать задачу для разработчика: создание ново...,создание_задачи,{'текст_задачи': 'создание новой функционально...
5,Создать заметку: идеи для рекламной кампании,создание_заметки,{'текст_заметки': 'идеи для рекламной кампании'}
6,Напомни мне о совещании с клиентом в четверг в...,напоминание,{'время_напоминания': 'четверг в 14:00'}
7,Создай задачу: подготовить презентацию для инв...,создание_задачи,{'текст_задачи': 'подготовить презентацию для ...
8,Запланируй встречу с командой на следующей неделе,планирование_встречи,{'дата_встречи': 'следующая неделя'}
9,Создай задачу: провести анализ рынка,создание_задачи,{'текст_задачи': 'провести анализ рынка'}


In [5]:
df_train = df['текстовое_сообщение'].to_frame().rename(columns={"текстовое_сообщение": "train_data"})

In [6]:
df_train

,train_data
0,Создай задачу: разработать дизайн логотипа
1,Создай заметку: идеи для нового проекта
2,Создай задачу: подготовить отчет к конференции
3,Создай заметку: список дел на неделю
4,Создать задачу для разработчика: создание ново...
5,Создать заметку: идеи для рекламной кампании
6,Напомни мне о совещании с клиентом в четверг в...
7,Создай задачу: подготовить презентацию для инв...
8,Запланируй встречу с командой на следующей неделе
9,Создай задачу: провести анализ рынка


In [7]:
def pattern_create(df_columns):
    pattern = []
    for i in df_columns:
        for k,v in i.items():
            pattern.append({"label" : k, "pattern" : v})
    return pattern
        

In [8]:
patterns = pattern_create(df['дополнительные_параметры'])

In [9]:
patterns[8]['pattern'] = "на следующей неделе"

In [10]:
patterns

[{'label': 'текст_задачи', 'pattern': 'разработать дизайн логотипа'},
 {'label': 'текст_заметки', 'pattern': 'идеи для нового проекта'},
 {'label': 'текст_задачи', 'pattern': 'подготовить отчет к конференции'},
 {'label': 'текст_заметки', 'pattern': 'список дел на неделю'},
 {'label': 'текст_задачи', 'pattern': 'создание новой функциональности'},
 {'label': 'текст_заметки', 'pattern': 'идеи для рекламной кампании'},
 {'label': 'время_напоминания', 'pattern': 'четверг в 14:00'},
 {'label': 'текст_задачи',
  'pattern': 'подготовить презентацию для инвесторов'},
 {'label': 'дата_встречи', 'pattern': 'на следующей неделе'},
 {'label': 'текст_задачи', 'pattern': 'провести анализ рынка'}]

In [11]:
nlp = spacy.blank("en")
ruler = nlp.add_pipe("entity_ruler")
ruler.add_patterns(patterns)

In [12]:
def convert_to_spacy(names_pattern, dataframe, path_out, spacy_names, spacy_ruler):
    print('START')
    
    def dataframe_predproc(pattern, dataframe, spacy_names, spacy_ruler):
        count =  0
        names = spacy_names
        ruler = spacy_ruler
        TRAIN_DATA = [] # наименование переменной TRAIN_DATA обязательна для перевода в spacy db
        for col in dataframe['train_data']:
            count +=1
            doc = names(str(col))
            print(doc.ents, count)
            entities = []
            for ent in doc.ents:
               # print(ruler.patterns[0:100])
             #   print(ent,ent.label_)
                entities.append([ent.start_char, ent.end_char, ent.label_])
            TRAIN_DATA.append([str(col), {"entities": entities}])
        return TRAIN_DATA
        
        
    def convert_to_db(lang: str, TRAIN_DATA, output_path: Path):
        nlp = spacy.blank(lang)
        db = DocBin()
        for text, annot in TRAIN_DATA:
            doc = nlp.make_doc(str(text))
            ents = []
            for start, end, label in annot["entities"]:
                span = doc.char_span(start, end, label=label)
                if span is None:
                    msg = f"Skipping entity [{start}, {end}, {label}] in the following text because the character span '{doc.text[start:end]}' does not align with token boundaries:\n\n{repr(text)}\n"
                    warnings.warn(msg)
                else:
                    ents.append(span)
            doc.ents = ents
            db.add(doc)
        return(db.to_disk(output_path))
    
    df = dataframe
    pattern = names_pattern
    names = spacy_names
    ruler = spacy_ruler
    TRAIN_DATA = dataframe_predproc(pattern, df, names, ruler)
    print(TRAIN_DATA[0:10])
    convert_to_db("en", TRAIN_DATA, str(path_out))

In [13]:
convert_to_spacy(patterns, df_train, SPACY_DATA_OUT_PATH + r'/jira_check.spacy',nlp, ruler)


START
(разработать дизайн логотипа,) 1
(идеи для нового проекта,) 2
(подготовить отчет к конференции,) 3
(список дел на неделю,) 4
(создание новой функциональности,) 5
(идеи для рекламной кампании,) 6
(четверг в 14:00,) 7
(подготовить презентацию для инвесторов,) 8
(на следующей неделе,) 9
(провести анализ рынка,) 10
[['Создай задачу: разработать дизайн логотипа', {'entities': [[15, 42, 'текст_задачи']]}], ['Создай заметку: идеи для нового проекта', {'entities': [[16, 39, 'текст_заметки']]}], ['Создай задачу: подготовить отчет к конференции', {'entities': [[15, 46, 'текст_задачи']]}], ['Создай заметку: список дел на неделю', {'entities': [[16, 36, 'текст_заметки']]}], ['Создать задачу для разработчика: создание новой функциональности', {'entities': [[33, 64, 'текст_задачи']]}], ['Создать заметку: идеи для рекламной кампании', {'entities': [[17, 44, 'текст_заметки']]}], ['Напомни мне о совещании с клиентом в четверг в 14:00', {'entities': [[37, 52, 'время_напоминания']]}], ['Создай зада